In [126]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [127]:
#page = driver.get("https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa")

page_url = 'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa'
page = requests.get(page_url)

In [128]:
content = page.content
soup = BeautifulSoup(content, features='html.parser')

span =soup.find_all('span')
print(len(span))

263


In [129]:
spans = soup.find_all("span", class_="css-rmqm02 eclomwz0")
addreses = soup.find_all("span", class_="css-17o293g es62z2j9")
spans[:4]

[<span class="css-rmqm02 eclomwz0">460 500 zł</span>,
 <span class="css-rmqm02 eclomwz0"><strong>10 858 zł/m²</strong></span>,
 <span class="css-rmqm02 eclomwz0">2 pokoje</span>,
 <span class="css-rmqm02 eclomwz0">42.41 m²</span>]

In [130]:
addreses[:5]

[<span class="css-17o293g es62z2j9">Warszawa, Ursus</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Białołęka, ul. Sprawna 7F</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Białołęka, Nowodwory, Mariana Wyrzykowskiego</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Mokotów, Puławska</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Bielany, Młociny, ul. Heroldów 10</span>]

In [131]:
spans[3:9]

[<span class="css-rmqm02 eclomwz0">42.41 m²</span>,
 <span class="css-rmqm02 eclomwz0">599 000 zł</span>,
 <span class="css-rmqm02 eclomwz0"><strong>11 745 zł/m²</strong></span>,
 <span class="css-rmqm02 eclomwz0">3 pokoje</span>,
 <span class="css-rmqm02 eclomwz0">51 m²</span>,
 <span class="css-rmqm02 eclomwz0">370 000 zł</span>]

In [132]:
price = spans[::4]
rooms = spans[2::4]
space = spans[3::4]

In [133]:
import re

def clean(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext  = re.sub(cleanr, '', str(raw_html))
    cleantext= re.sub("[^0-9]", "", cleantext)
    return cleantext 

cleaned_price = []

for p in price:
    cleaned = clean(p)
    cleaned_price.append(cleaned)
    
cleaned_price[:5]

['460500', '599000', '370000', '639000', '875000']

In [134]:
[clean(p) for p in price][:5]

['460500', '599000', '370000', '639000', '875000']

In [135]:
cleaned_rooms =[]

for r in rooms:
    cleaned = clean(r)
    cleaned_rooms.append(cleaned)
cleaned_rooms[:5]

['2', '3', '2', '3', '3']

In [136]:
space[:5]

[<span class="css-rmqm02 eclomwz0">42.41 m²</span>,
 <span class="css-rmqm02 eclomwz0">51 m²</span>,
 <span class="css-rmqm02 eclomwz0">32.7 m²</span>,
 <span class="css-rmqm02 eclomwz0">48 m²</span>,
 <span class="css-rmqm02 eclomwz0">70 m²</span>]

In [137]:
cleaned_space =[]

for s in space:
    s = str(s)
    cleaned_space.append(s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ',''))

cleaned_space[:5]

['42.41', '51', '32.7', '48', '70']

In [138]:
addreses[:5]

[<span class="css-17o293g es62z2j9">Warszawa, Ursus</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Białołęka, ul. Sprawna 7F</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Białołęka, Nowodwory, Mariana Wyrzykowskiego</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Mokotów, Puławska</span>,
 <span class="css-17o293g es62z2j9">Warszawa, Bielany, Młociny, ul. Heroldów 10</span>]

In [139]:
list = []

for adr in addreses:
    a = str(adr)
    list.append(a.replace('<span class="css-17o293g es62z2j9">', '').replace('</span>', ''))

In [140]:
list[:5]

['Warszawa, Ursus',
 'Warszawa, Białołęka, ul. Sprawna 7F',
 'Warszawa, Białołęka, Nowodwory, Mariana Wyrzykowskiego',
 'Warszawa, Mokotów, Puławska',
 'Warszawa, Bielany, Młociny, ul. Heroldów 10']

In [141]:
df = pd.DataFrame({'Address': list, 'Price': cleaned_price, 'Space': cleaned_space, 'Rooms': cleaned_rooms})
df

,Address,Price,Space,Rooms
0,"Warszawa, Ursus",460500,42.41,2
1,"Warszawa, Białołęka, ul. Sprawna 7F",599000,51,3
2,"Warszawa, Białołęka, Nowodwory, Mariana Wyrzyk...",370000,32.7,2
3,"Warszawa, Mokotów, Puławska",639000,48,3
4,"Warszawa, Bielany, Młociny, ul. Heroldów 10",875000,70,3
5,"Warszawa, Praga-Południe, ul. Grochowska",563000,47,2
6,"Warszawa, Praga-Północ, Jagiellońska",599000,39,2
7,"Warszawa, Bemowo, Chrzanów",508850,35.89,2
8,"Warszawa, Ursynów, Puławska",639000,48,3
9,"Warszawa, Śródmieście, Stanisława Dubois",415000,28.1,1


In [142]:
df.Address[2]

'Warszawa, Białołęka, Nowodwory, Mariana Wyrzykowskiego'

In [143]:
"https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=" + str(1)

'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1'

In [144]:
def cleaned_space(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    s = str(string)
    return (s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ','')) 

def clean_without_coma(raw_html):
   """clean numbers froim html 

   Args:
       raw_html (beautiful suoup object): _description_

   Returns:
       int: scrapped number
   """
   cleanr = re.compile('<.*?>')
   cleantext  = re.sub(cleanr, '', str(raw_html))
   cleantext= re.sub("[^0-9]", "", cleantext)
   return cleantext 
   
def cleaned_addreses(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    a = str(string)
    return a.replace('<span class="css-17o293g es62z2j9">', '').replace('</span>', '')

In [145]:
addreses=[] #List of flats adresses
out_prices=[] #List house prices
out_space=[] #List of house space
out_rooms=[] #List of number of rooms in flat

page_url = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page="

for number in range(5):
    page = requests.get(page_url +str(number))
    content = page.content
    soup = BeautifulSoup(content, features='html.parser')

    spans = soup.find_all("span", class_="css-rmqm02 eclomwz0")
    addres = soup.find_all("span", class_="css-17o293g es62z2j9")
    price = spans[::4]
    rooms = spans[2::4]
    space = spans[3::4]

    clean_price = [clean(p) for p in price]
    clean_rooms = [clean(r) for r in rooms]
    clean_space = [cleaned_space(s) for s in space]
    clean_addres = [cleaned_addreses(a) for a in addres]

    addreses += clean_addres
    out_prices += clean_price
    out_space += clean_space
    out_rooms += clean_rooms






In [146]:
cleaned_space(space[0])

'52.9'

In [147]:
clean_addres[:5]

['Warszawa, Białołęka, Odkryta 48D',
 'Warszawa, Praga-Południe, Gocławek, ul. Makowska 33',
 'Warszawa, Śródmieście, Śródmieście Południowe, Wspólna 52/54',
 'Warszawa, Bielany',
 'Warszawa, Wola, Jana Kazimierza']

In [148]:
clean_space[:5]

['52.9', '36.1', '40', '30', '33.13']

In [149]:
df = pd.DataFrame({'Address': addreses, 'Price': out_prices, 'Space': out_space, 'Rooms': out_rooms})
#df.to_csv('listings.csv', index=False, encoding='utf-8')

In [150]:
df

,Address,Price,Space,Rooms
0,"Warszawa, Praga-Południe, Gocław, ul. Brygady ...",549000,42.1,2
1,"Warszawa, Bielany, ul. Pabla Nerudy",699000,62,3
2,"Warszawa, Białołęka, ul. Kąty Grodziskie",599000,55,3
3,"Warszawa, Mokotów, Puławska",639000,48,3
4,"Warszawa, Bielany, Młociny, ul. Heroldów 10",875000,70,3
...,...,...,...,...
190,"Warszawa, Targówek, ul. Janowiecka",,62.82,4
191,"Warszawa, Wola, ul. Ogrodowa",,247.69,6
192,"Warszawa, Ursynów, ul. Kabacki Dukt",1727645,101.92,3
193,"Warszawa, Bielany, ul. Improwizacji",628000,46.24,2


In [151]:
df[:5]

,Address,Price,Space,Rooms
0,"Warszawa, Praga-Południe, Gocław, ul. Brygady ...",549000,42.1,2
1,"Warszawa, Bielany, ul. Pabla Nerudy",699000,62,3
2,"Warszawa, Białołęka, ul. Kąty Grodziskie",599000,55,3
3,"Warszawa, Mokotów, Puławska",639000,48,3
4,"Warszawa, Bielany, Młociny, ul. Heroldów 10",875000,70,3


In [152]:
soup.find_all("a", href = True)[1]

<a class="css-zygftp e1ia8j2v0" href="https://pomoc.otodom.pl/hc/pl/articles/203136942-Co-wp%C5%82ywa-na-kolejno%C5%9B%C4%87-og%C5%82osze%C5%84-w-wynikach-wyszukiwania-" rel="noreferrer" target="_blank">Jak pozycjonowane są ogłoszenia?</a>

In [153]:
inner_page_url = "https://www.otodom.pl"

for a in soup.find_all('a', href=True):
    print(inner_page_url + a['href'])

https://www.otodom.pl/
https://www.otodom.plhttps://pomoc.otodom.pl/hc/pl/articles/203136942-Co-wp%C5%82ywa-na-kolejno%C5%9B%C4%87-og%C5%82osze%C5%84-w-wynikach-wyszukiwania-
https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&isPromoted=true
https://www.otodom.pl/pl/oferta/2-pokoje-13-min-do-metra-garaz-i-komorka-ID4h80u
https://www.otodom.pl/pl/oferta/mieszkanie-2-pokoje-kuchnia-lazienka-36-1m2-goclaw-ID4fZ11
https://www.otodom.pl/pl/oferta/bezposrednio-dwa-pokoje-ul-wspolna-ID4hfaQ
https://www.otodom.pl/pl/oferta/dostepne-bezposrednio-nowe-wykonczone-mieszkanie-ID4gGEr
https://www.otodom.pl/pl/oferta/ogrodek-z-tarasem-szybki-dojazd-do-centrum-ID4hjyP
https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-44m2-balkon-ID4dxz7
https://www.otodom.pl/pl/oferta/mieszkanie-na-biuro-metro-stare-bielany-ID4h2md
https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-30m2-bezposrednio-ID4gN0q
https://www.otodom.pl/pl/oferta/2-pokojo

In [154]:
inner_page_url = "https://www.otodom.pl"

In [155]:
a = soup.find_all('a', href=True)[2:41]

In [156]:
type(soup)

bs4.BeautifulSoup

In [157]:
a[5]['href']

'/pl/oferta/ogrodek-z-tarasem-szybki-dojazd-do-centrum-ID4hjyP'

In [158]:
for a in soup.find_all('a', href=True)[2:41]:
    print(inner_page_url + a['href'])

https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&isPromoted=true
https://www.otodom.pl/pl/oferta/2-pokoje-13-min-do-metra-garaz-i-komorka-ID4h80u
https://www.otodom.pl/pl/oferta/mieszkanie-2-pokoje-kuchnia-lazienka-36-1m2-goclaw-ID4fZ11
https://www.otodom.pl/pl/oferta/bezposrednio-dwa-pokoje-ul-wspolna-ID4hfaQ
https://www.otodom.pl/pl/oferta/dostepne-bezposrednio-nowe-wykonczone-mieszkanie-ID4gGEr
https://www.otodom.pl/pl/oferta/ogrodek-z-tarasem-szybki-dojazd-do-centrum-ID4hjyP
https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-44m2-balkon-ID4dxz7
https://www.otodom.pl/pl/oferta/mieszkanie-na-biuro-metro-stare-bielany-ID4h2md
https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-30m2-bezposrednio-ID4gN0q
https://www.otodom.pl/pl/oferta/2-pokojowe-mieszkanie-38m2-loggia-ID4fYAa
https://www.otodom.pl/pl/oferta/sadyba-przytulne-i-przestronne-2-pokoje-ID4hso0
https://www.otodom.pl/pl/oferta/12min-do-metra-promocja-do-5-

In [175]:
url = 'https://www.otodom.pl/pl/oferta/2-pokoje-13-min-do-metra-garaz-i-komorka-ID4h80u'

page = requests.get(url)
content = page.content
soup = BeautifulSoup(content, features='html.parser')

data= soup.find_all("div", class_="css-1wi2w6s estckra5")

market = data[10]
year = data[12]
elevator = data[15]
balcony = data[5]
level = data[4]
parking_place =data[7]

In [177]:
data[:15]

[<div class="css-1wi2w6s estckra5">52,9 m²</div>,
 <div class="css-1wi2w6s estckra5">pełna własność</div>,
 <div class="css-1wi2w6s estckra5">2 </div>,
 <div class="css-1wi2w6s estckra5">do zamieszkania</div>,
 <div class="css-1wi2w6s estckra5">4/6</div>,
 <div class="css-1wi2w6s estckra5">balkon</div>,
 <div class="css-1wi2w6s estckra5">700 zł</div>,
 <div class="css-1wi2w6s estckra5">garaż/miejsce parkingowe</div>,
 <div class="css-1wi2w6s estckra5">tak</div>,
 <div class="css-1wi2w6s estckra5">miejskie</div>,
 <div class="css-1wi2w6s estckra5">wtórny</div>,
 <div class="css-1wi2w6s estckra5">biuro nieruchomości </div>,
 <div class="css-1wi2w6s estckra5">2003</div>,
 <div class="css-1wi2w6s estckra5">blok</div>,
 <div class="css-1wi2w6s estckra5">drewniane</div>]

In [198]:
print(year)
clean(year)

<div class="css-1wi2w6s estckra5">2003</div>


'2003'

In [199]:
print(level)
clean(level)

<div class="css-1wi2w6s estckra5">4/6</div>


'46'

In [214]:
str(data[4])[34:-6]

'4/6'

In [201]:
print(market)
clean(market)

<div class="css-1wi2w6s estckra5">wtórny</div>


''

In [215]:
str(data[10])[34:-6]

'wtórny'

In [200]:
print(elevator)
clean(elevator)

<div class="css-1wi2w6s estckra5">tak</div>


''

In [216]:
str(data[15])[34:-6]

'tak'

In [217]:
str(data[5])[34:-6]

'balkon'

In [218]:
str(data[7])[34:-6]

'garaż/miejsce parkingowe'

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re


def clean(raw_html):
   """clean numbers froim html 

   Args:
       raw_html (beautiful suoup object): _description_

   Returns:
       int: scrapped number
   """
   cleanr = re.compile('<.*?>')
   cleantext  = re.sub(cleanr, '', str(raw_html))
   cleantext= re.sub("[^0-9]", "", cleantext)
   return cleantext 

def cleaned_addreses(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    a = str(string)
    return a.replace('<span class="css-17o293g es62z2j9">', '').replace('</span>', '')


def cleaned_space(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    s = str(string)
    return (s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ','')) 

def cleaned_elevator(string):
    """_summary_

    Args:
        string (_type_): _description_

    Returns:
        _type_: _description_
    """
    s = str(string)
    return (s[34:40].replace('m','').replace('<',"").replace('²','').replace(' ','')) 

def scrap_data():
    """scrap data

    Returns:
        pandas_data_frame: four colums
    """
    addreses = [] #List of flats adresses
    out_prices = [] #List house prices
    out_space = [] #List of house space
    out_rooms = [] #List of number of rooms in flat
    out_market = []
    out_year = []
    out_elevator = []
    out_balcony = []
    out_level = []
    out_parking_place = []

    page_url = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page="

    for number in range(5):
        page = requests.get(page_url +str(number))
        content = page.content
        soup = BeautifulSoup(content, features='html.parser')

        spans = soup.find_all("span", class_="css-rmqm02 eclomwz0")
        addres = soup.find_all("span", class_="css-17o293g es62z2j9")
        price = spans[::4]
        rooms = spans[2::4]
        space = spans[3::4]

        clean_price = [clean(p) for p in price]
        clean_rooms = [clean(r) for r in rooms]
        clean_space = [cleaned_space(s) for s in space]
        clean_addres = [cleaned_addreses(a) for a in addres]

        market, year, elevator, balcony, level, parking_place = inner_page_data_scrap(soup)

        addreses += clean_addres
        out_prices += clean_price
        out_space += clean_space
        out_rooms += clean_rooms
        out_market += market
        out_year += year
        out_elevator += elevator
        out_balcony += balcony
        out_level += level
        out_parking_place += parking_place

    df = pd.DataFrame({
        'Address': addreses, 'Price': out_prices, 'Space': out_space, 'Rooms': out_rooms, 'Market': out_market, 'Year': out_year, 'Evelevator': out_elevator, 'Balcony': out_balcony, 'Level': out_level, 'Parking place': out_parking_place
        })
    return df

def inner_page_data_scrap(soup):
    """scraps detail from inner html page of an offert

    Args:
        soup (bs4.BeautifulSoup): current html page of offerts list

    Returns:
       market, year, elevator, balcony, level, parking_place (list): _description_
    """

    inner_page_url = "https://www.otodom.pl"
    market = []
    year = []
    elevator = []
    balcony = []
    level = []
    parking_place = []

    for a in soup.find_all('a', href=True)[2:41]:
        pages_url = inner_page_url + a['href']

    for u in pages_url:
        if len(u) < 5:
            pass 
        else:
            page = requests.get(u)
            content = page.content
            soup = BeautifulSoup(content, features='html.parser')

        data= soup.find_all("div", class_="css-1wi2w6s estckra5")

        market += str(data[10])[34:-6]
        year += clean(data[12])
        elevator += str(data[15])[34:-6]
        balcony += str(data[5])[34:-6]
        level += str(data[4])[34:-6]
        parking_place += str(data[5])[34:-6]

    return market, year, elevator, balcony, level, parking_place


data = scrap_data()

IndexError: list index out of range

In [ ]:
data

In [12]:
inner_page_url = "https://www.otodom.pl"
page_url = "https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page="
number =1
page = requests.get(page_url +str(number))
content = page.content
soup = BeautifulSoup(content, features='html.parser')

for a in soup.find_all('a', href=True)[2:41]:
    print(inner_page_url + a['href'])

https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page=1&limit=36&market=ALL&locations=%5Bcities_6-26%5D&isPromoted=true
https://www.otodom.pl/pl/oferta/sprzedam-bezposrednio-mieszkanie-w-samym-centrum-ID4hkLT
https://www.otodom.pl/pl/oferta/nowe-dwupokojowe-mieszkanie-z-loggia-ID4gQoW
https://www.otodom.pl/pl/oferta/bezposrednio-ostatnie-pietro-lux-2-pok-ID4hfCK
https://www.otodom.pl/pl/oferta/jasne-nowoczesne-mieszkanie-na-bemowie-metro-ID4himm
https://www.otodom.pl/pl/oferta/mieszkanie-na-zielonym-bemowie-spokojna-okolica-ID4heyQ
https://www.otodom.pl/pl/oferta/mieszkanie-po-remoncie-na-osiedlu-lewandow-park-2-ID4hcbX
https://www.otodom.pl/pl/oferta/nowoczesna-inwestycja-na-warszawskiej-woli-ID4haat
https://www.otodom.pl/pl/oferta/doskonala-inwestycja-na-bemowie-blisko-metra-ID4haar
https://www.otodom.pl/pl/oferta/mieszkanie-idealne-dla-mlodej-rodziny-bielany-38m2-ID4gY0p
https://www.otodom.pl/pl/oferta/przyszlosciowa-inwestycja-na-bemowie-blisko-metra-ID4h6YQ
https://www

In [10]:
pages_url

'https://www.otodom.pl/pl/oferta/klimatyczne-mieszkanie-zakup-z-najemca-lub-beze-ID4h9Bu'